In [2]:
import olca_schema as o
import olca_ipc as ipc
    

# Connect to ipc
client = ipc.Client(8080)    
print("\n✓ Connected to openLCA IPC server")




✓ Connected to openLCA IPC server


In [28]:
# Verify connection by getting database info
flows = list(client.get_descriptors(o.Flow))
print(f"Connected! Database contains {len(flows)} flows.")

Connected! Database contains 63948 flows.


# ----------------------------------------------------------------------------
# Search for Background Processes
# ----------------------------------------------------------------------------


In [3]:
def find_flow_by_keywords(keywords, max_results=5):
    """
    Search flows using partial keyword matching.
    
    Args:
        keywords: list of keywords (case-insensitive)
        max_results: maximum number of results to show
    
    Returns:
        list of matching flow references
    """
    matches = []
    keywords_lower = [k.lower() for k in keywords]
    
    print(  f"  Searching: {' + '.join(keywords)}")
    
    for flow_ref in client.get_descriptors(o.Flow):
        name_lower = flow_ref.name.lower()
        
        # Check if ALL keywords are in the flow name
        if all(kw in name_lower for kw in keywords_lower):
            matches.append(flow_ref)
            if len(matches) <= max_results:
                print(f"    → {flow_ref.name}")
    
    if len(matches) > max_results:
        print(f"    ... and {len(matches) - max_results} more matches")
    elif len(matches) == 0:
        print(f"    ✗ No matches found")
    
    return matches

In [5]:
find_flow_by_keywords(['electricity'])

  Searching: electricity
    → electricity, low voltage
    → electricity, medium voltage
    → electricity, high voltage
    → distribution network, electricity, low voltage
    → electricity, for reuse in municipal waste incineration only
    ... and 37 more matches


[Ref(id='d69294d7-8d64-4915-a896-9996a014c410', category='D:Electricity, gas, steam and air conditioning supply/35:Electricity, gas, steam and air conditioning supply/351:Electric power generation, transmission and distribution/3510:Electric power generation, transmission and distribution', description=None, flow_type=<FlowType.PRODUCT_FLOW: 'PRODUCT_FLOW'>, library=None, location=None, name='electricity, low voltage', process_type=None, ref_unit='MJ', ref_type=<RefType.Flow: 'Flow'>),
 Ref(id='759b89bd-3aa6-42ad-b767-5bb9ef5d331d', category='D:Electricity, gas, steam and air conditioning supply/35:Electricity, gas, steam and air conditioning supply/351:Electric power generation, transmission and distribution/3510:Electric power generation, transmission and distribution', description=None, flow_type=<FlowType.PRODUCT_FLOW: 'PRODUCT_FLOW'>, library=None, location=None, name='electricity, medium voltage', process_type=None, ref_unit='MJ', ref_type=<RefType.Flow: 'Flow'>),
 Ref(id='66c93e

In [6]:
def find_best_provider(flow_ref):
    """Get the first (best) provider process for a flow"""
    # Handle both method names for compatibility
    try:
        if hasattr(client, 'get_providers'):
            providers = list(client.get_providers(flow_ref))
        else:
            providers = list(client.get_providers_of(flow_ref))
    except Exception as e:
        print(f"    ⚠ Error getting providers: {e}")
        return None
    
    if providers:
        # TechFlow has 'provider' attribute which is the process Ref
        first = providers[0]
        if hasattr(first, 'provider'):
            return first.provider  # Return the process Ref
        elif hasattr(first, 'process'):
            return first.process  # Alternative attribute name
        else:
            return first  # Fallback
    return None



In [12]:
materials_flows = {}
materials_providers = {}
matches =find_flow_by_keywords(['electricity'])
materials_flows['electricity'] = matches[0]
materials_providers['electricity'] =find_best_provider(materials_flows[0])
print(materials_providers)



    

  Searching: electricity
    → electricity, low voltage
    → electricity, medium voltage
    → electricity, high voltage
    → distribution network, electricity, low voltage
    → electricity, for reuse in municipal waste incineration only
    ... and 37 more matches


KeyError: 0

In [8]:
# Store materials
materials_flows = {}
materials_providers = {}

# Water
print("7. Water (Tap Water)")
matches = find_flow_by_keywords(['tap', 'water'])
if not matches:
    matches = find_flow_by_keywords(['water', 'decarbonised'])
if matches:
    materials_flows['Water'] = matches[0]
    materials_providers['Water'] = find_best_provider(matches[0])
    if materials_providers['Water']:
        prov = materials_providers['Water']
        prov_name = prov.name if hasattr(prov, 'name') else str(prov)
        print(f"  ✓ Provider: {prov_name}\n")
    else:
        print(f"   No provider found\n")
else:
    print("   Not found\n")

7. Water (Tap Water)
  Searching: tap + water
    → tap water
  ✓ Provider: market for tap water | tap water | Cutoff, U




# Creating Product Flows



In [18]:


mass_prop = client.get(o.FlowProperty, name="Mass")



def create_flow(name):
    """Create a product flow"""
    flow = o.Flow()
    flow.name = name
    flow.flow_type = o.FlowType.PRODUCT_FLOW
    
    flow.flow_properties = [
        o.FlowPropertyFactor(
            flow_property=o.Ref(
                id=mass_prop.id,
                name=mass_prop.name,
                ref_type=o.RefType.FlowProperty
            ),
            conversion_factor=1.0,
            is_ref_flow_property=True
        )
    ]
    
    client.put(flow)
    print(f"  ✓ {name}")
    return flow

In [19]:
create_flow("TTestflow")

  ✓ TTestflow


Flow(id='1269b203-a371-43ad-8816-f7aecaaa1e16', cas=None, category=None, description=None, flow_properties=[FlowPropertyFactor(conversion_factor=1.0, flow_property=Ref(id='93a60a56-a3c8-11da-a746-0800200b9a66', category=None, description=None, flow_type=None, library=None, location=None, name='Mass', process_type=None, ref_unit=None, ref_type=<RefType.FlowProperty: 'FlowProperty'>), is_ref_flow_property=True)], flow_type=<FlowType.PRODUCT_FLOW: 'PRODUCT_FLOW'>, formula=None, is_infrastructure_flow=None, last_change='2025-11-08T17:02:46.602805+00:00', library=None, location=None, name='TTestflow', synonyms=None, tags=None, version='01.00.000')

In [ ]:
# PET flows
pet_mix = create_flow("PET granulate mix")
pet_mix_trans = create_flow("PET granulate mix, transported")
pet_bottle = create_flow("PET bottle, filled")


In [ ]:

# PC flows
pc_mix = create_flow("PC granulate mix")
pc_mix_trans = create_flow("PC granulate mix, transported")
pc_bottle = create_flow("PC bottle, filled")


## Create Processes

In [ ]:

print("\n--- Creating Processes ---")

def make_exchange(flow_ref, amount, is_input, is_qref=False, provider_ref=None):
    """Create an exchange with proper linking
    Exchanges are the inputs to a process
    """
    ex = o.Exchange()
    
    # Handle flow reference
    if isinstance(flow_ref, o.Ref):
        ex.flow = flow_ref
    elif isinstance(flow_ref, o.Flow):
        ex.flow = o.Ref(id=flow_ref.id, name=flow_ref.name, ref_type=o.RefType.Flow)
    elif hasattr(flow_ref, 'id'):
        ex.flow = o.Ref(id=flow_ref.id, name=flow_ref.name, ref_type=o.RefType.Flow)
    
    ex.amount = amount
    ex.unit = o.Ref(id=kg_unit.id, name=kg_unit.name, ref_type=o.RefType.Unit)
    ex.flow_property = o.Ref(id=mass_prop.id, name=mass_prop.name, ref_type=o.RefType.FlowProperty)
    ex.is_input = is_input
    ex.is_quantitative_reference = is_qref
    
    # Link provider
    if provider_ref:
        if isinstance(provider_ref, o.Ref):
            ex.default_provider = provider_ref
        elif hasattr(provider_ref, 'id'):
            ex.default_provider = o.Ref(
                id=provider_ref.id,
                name=provider_ref.name,
                ref_type=o.RefType.Process
            )
    
    return ex

In [ ]:

def create_process(name, description, exchanges):
    """Create a process"""
    process = o.Process()
    process.name = name
    process.description = description
    process.process_type = o.ProcessType.UNIT_PROCESS
    
    for i, ex in enumerate(exchanges, start=1):
        ex.internal_id = i
    
    process.exchanges = exchanges
    process.last_internal_id = len(exchanges)
    
    client.put(process)
    print(f"  ✓ {name}")
    return process


In [ ]:

# PET Production Chain
print("\nPET Production Chain:")

# PET Granulate Production
#creating inputs to pet production
ex_pet_prod = [make_exchange(pet_mix, 0.065, False, True)]
if 'PET' in materials_flows:
    ex_pet_prod.append(make_exchange(materials_flows['PET'], 0.060, True, 
                                     provider_ref=materials_providers.get('PET')))
if 'HDPE' in materials_flows:
    ex_pet_prod.append(make_exchange(materials_flows['HDPE'], 0.004, True,
                                     provider_ref=materials_providers.get('HDPE')))
if 'PP' in materials_flows:
    ex_pet_prod.append(make_exchange(materials_flows['PP'], 0.001, True,
                                     provider_ref=materials_providers.get('PP')))



pet_prod = create_process(
    "PET Granulate Production",
    "Mixing PET (60g) + HDPE (4g) + PP (1g)",
    ex_pet_prod
)



In [ ]:

# PET Transport
# Transporting 65g of PET mix over 500 km by lorry
# add transport freight input

ex_pet_trans =[
    
        make_exchange(pet_mix_trans, 0.065, False, True),
        make_exchange(pet_mix, 0.065, True, provider_ref=pet_prod),
        #for freight input, uncomment below
        # make_exchange(trans_freight_lorry, 0.065*500, True, provider_ref=trans_freight_lorry)
    
]
if "transport" in materials_flows:
    ex_pet_trans.append(
        make_exchange(
            materials_flows["transport"],
            0.065 * 500,
            True,
            provider_ref=materials_providers.get("transport")
        )
    )

pet_trans = create_process(
    "PET Granulate Transport",
    "Transport 500 km by lorry",
    ex_pet_trans
)






In [ ]:

# PET Filling
# The 
ex_pet_fill = [
    make_exchange(pet_bottle, 1.065, False, True),
    make_exchange(pet_mix_trans, 0.065, True, provider_ref=pet_trans)
]
if 'Water' in materials_flows:
    ex_pet_fill.append(make_exchange(materials_flows['Water'], 1.0, True,
                                     provider_ref=materials_providers.get('Water')))

pet_fill = create_process(
    "PET Bottle Filling",
    "Filling with 1L water",
    ex_pet_fill
)

In [ ]:

# PC Production Chain
print("\nPC Production Chain:")

# PC Granulate Production
ex_pc_prod = [make_exchange(pc_mix, 0.065, False, True)]
if 'PC' in materials_flows:
    ex_pc_prod.append(make_exchange(materials_flows['PC'], 0.060, True,
                                    provider_ref=materials_providers.get('PC')))
if 'LDPE' in materials_flows:
    ex_pc_prod.append(make_exchange(materials_flows['LDPE'], 0.004, True,
                                    provider_ref=materials_providers.get('LDPE')))
if 'PB' in materials_flows:
    ex_pc_prod.append(make_exchange(materials_flows['PB'], 0.001, True,
                                    provider_ref=materials_providers.get('PB')))

pc_prod = create_process(
    "PC Granulate Production",
    "Mixing PC (60g) + LDPE (4g) + PB (1g)",
    ex_pc_prod
)

# PC Transport
ex_pc_trans=[
 
        make_exchange(pc_mix_trans, 0.065, False, True),
        make_exchange(pc_mix, 0.065, True, provider_ref=pc_prod)
  
]
if "transport" in materials_flows:
    ex_pc_trans.append(
        make_exchange(
            materials_flows["transport"],
            0.065 * 500,
            True,
            provider_ref=materials_providers.get("transport")
        )
    )
pc_trans = create_process(
    "PC Granulate Transport",
    "Transport 500 km by lorry",
    ex_pc_trans
)

# PC Filling
ex_pc_fill = [
    make_exchange(pc_bottle, 1.065, False, True),
    make_exchange(pc_mix_trans, 0.065, True, provider_ref=pc_trans)
]
if 'Water' in materials_flows:
    ex_pc_fill.append(make_exchange(materials_flows['Water'], 1.0, True,
                                    provider_ref=materials_providers.get('Water')))

pc_fill = create_process(
    "PC Bottle Filling",
    "Filling with 1L water",
    ex_pc_fill
)


## Creating Product Systems


In [ ]:

print("\n--- Creating Product Systems ---")
# create_product_sytem is an existing method in the client
# Pass the process object, not just the ID
pet_system = client.create_product_system(pet_fill)
if pet_system:
    print(f"  ✓ PET System: {pet_system.name}")
else:
    print("  ✗ Failed to create PET system")
    exit(1)

pc_system = client.create_product_system(pc_fill)
if pc_system:
    print(f"  ✓ PC System: {pc_system.name}")
else:
    print("  ✗ Failed to create PC system")
    exit(1)



 ============================================================================
# PHASE 3: IMPACT ASSESSMENT
 ============================================================================


In [27]:
# Find LCIA method
print("\n--- Finding LCIA Method ---")
method = None
# Set this to a method name you prefer (e.g. 'ReCiPe') or None to skip",
ReCiPe"  # change to None to skip preferred lookup
# Build preferred order: try user-chosen first, then common fallbacks
preferred = []
if impact_method:
    preferred.append(impact_method)
for name in ['TRACI', 'ReCiPe', 'ILCD', 'CML', 'EF']:
    if name not in preferred:
        preferred.append(name)
# Try preferred methods in order
for name in preferred:
    try:
        method = client.get(o.ImpactMethod, name=name)
    except Exception:
        method = None
    if method:
        print(f"  ✓ Using: {method.name}")
        break

if not method:
    print("  ⚠ No preferred method found. Listing available methods...")
    method_refs = list(client.get_descriptors(o.ImpactMethod))
    if not method_refs:
        print("  ✗ No LCIA method available")
        exit(1)
    # Print available methods and let the user choose
    for i, ref in enumerate(method_refs, start=1):
        print(f"  {i}: {ref.name}")
    sel = input("Select method by number (or press Enter to use first): ").strip()
    try:
        if sel:
            idx = int(sel) - 1
            chosen = method_refs[idx]
        else:
            chosen = method_refs[0]
    except Exception as e:
        print(f"  ⚠ Invalid selection ({e}), using first available")
        chosen = method_refs[0]
    method = client.get(o.ImpactMethod, chosen.id)
    print(f"  ✓ Using: {method.name}")


SyntaxError: unterminated string literal (detected at line 5) (1921420670.py, line 5)

In [ ]:
# Calculate
print("\n--- Calculating ---")

setup_pet = o.CalculationSetup()
setup_pet.target = pet_system
setup_pet.impact_method = method.to_ref()
setup_pet.amount = 1.0

result_pet = client.calculate(setup_pet)
result_pet.wait_until_ready()
print("  ✓ PET complete")

setup_pc = o.CalculationSetup()
setup_pc.target = pc_system
setup_pc.impact_method = method.to_ref()
setup_pc.amount = 1.0

result_pc = client.calculate(setup_pc)
result_pc.wait_until_ready()
print("  ✓ PC complete")


# PHASE 4: INTERPRETATION

In [ ]:

print("\n" + "="*70)
print("PHASE 4: INTERPRETATION")
print("="*70)

print("\n--- Comparative Results ---\n")
print(f"{'Impact Category':<50} {'PET':<18} {'PC':<18} {'Difference':<15}")
print("-" * 100)

try:
    impacts_pet = result_pet.get_total_impacts()
    impacts_pc = result_pc.get_total_impacts()
    
    if impacts_pet and impacts_pc:
        data = []
        
        for i, imp_pet in enumerate(impacts_pet):
            if i < len(impacts_pc):
                imp_pc = impacts_pc[i]
                
                name = imp_pet.impact_category.name[:49]
                v_pet = imp_pet.amount if hasattr(imp_pet, 'amount') else (imp_pet.value if hasattr(imp_pet, 'value') else 0)
                v_pc = imp_pc.amount if hasattr(imp_pc, 'amount') else (imp_pc.value if hasattr(imp_pc, 'value') else 0)
                
                diff_pct = ((v_pc - v_pet) / abs(v_pet) * 100) if abs(v_pet) > 1e-10 else 0
                diff_str = f"{diff_pct:+.1f}%" if abs(v_pet) > 1e-10 else "N/A"
                
                data.append({'name': name, 'pet': v_pet, 'pc': v_pc, 'diff': diff_pct})
                print(f"{name:<50} {v_pet:<18.4e} {v_pc:<18.4e} {diff_str:<15}")
        
        if data:
            # Summary
            print("\n" + "="*70)
            print("KEY FINDINGS")
            print("="*70)
            
            total_pet = sum(abs(d['pet']) for d in data)
            total_pc = sum(abs(d['pc']) for d in data)
            
            print(f"\nTotal Normalized Impact:")
            print(f"  PET: {total_pet:.4e}")
            print(f"  PC:  {total_pc:.4e}")
            
            pet_better = sum(1 for d in data if d['diff'] > 0)
            pc_better = sum(1 for d in data if d['diff'] < 0)
            
            print(f"\nCategory Comparison:")
            print(f"  PET better: {pet_better}/{len(data)}")
            print(f"  PC better:  {pc_better}/{len(data)}")
            
            if total_pet < total_pc:
                pct = (total_pc - total_pet) / total_pet * 100
                print(f"\n✓ PET shows {pct:.1f}% lower overall impact")
                winner = "PET"
            elif total_pc < total_pet:
                pct = (total_pet - total_pc) / total_pc * 100
                print(f"\n✓ PC shows {pct:.1f}% lower overall impact")
                winner = "PC"
            else:
                print(f"\n≈ Similar overall impact")
                winner = "Neither"
            
            print("\n" + "="*70)
            print("RECOMMENDATIONS")
            print("="*70)
            print(f"""
1. Material Selection: {'Consider ' + winner if winner != 'Neither' else 'Both viable'}
2. End-of-Life: Evaluate recycling infrastructure
3. Lightweighting: Reduce material use
4. Supply Chain: Optimize transportation
5. Full LCA: Extend to use phase & disposal
            """)
        else:
            print("\n⚠ No impact data available")
    else:
        print("  ⚠ No impact results")
        
except Exception as e:
    print(f"  ✗ Error: {e}")
    import traceback
    traceback.print_exc()

# Cleanup
try:
    result_pet.dispose()
    result_pc.dispose()
    print("\n✓ Results disposed")
except:
    pass

print("\n" + "="*70)
print("LCA STUDY COMPLETE")
print("="*70)
print("\n✓ Check openLCA UI for detailed process network")